In [ ]:
%load_ext watermark

import xarray as xr
import xproj
from geoindex import with_geoindex
from raster_index import RasterIndex

%watermark -iv
import numpy as np

ds = xr.open_dataset(
    # "/Users/deepak/repos/rioxarray/test/test_data/input/PLANET_SCOPE_3D.nc",
    "./S_20240101_concentration_v3.0.tif",
    # decode_coords="all",
    engine="rasterio",
    backend_kwargs={"parse_coordinates": False},
)
ds

In [ ]:
varname = "spatial_ref"
var = ds["spatial_ref"]
x_dim = "x"
y_dim = "y"

from affine import Affine

if geotransform := var.attrs.get("GeoTransform"):
    # TODO: delete GeoTransform if successful?
    geotransform = np.fromstring(geotransform, sep=" ").tolist()
fwd = Affine.from_gdal(*geotransform[:6])
centers = fwd * fwd.translation(0.5, 0.5)
index = RasterIndex.from_transform(
    affine=centers,
    width=ds.sizes.get(x_dim),
    height=ds.sizes.get(y_dim),
    x_dim=x_dim,
    y_dim=y_dim,
)

newds = ds.set_xindex(["spatial_ref"], xproj.CRSIndex).assign_coords(
    xr.Coordinates.from_xindex(index)
)
newds

In [ ]:
xr.concat(
    [
        newds.isel(x=slice(10, 20)),
        newds.isel(x=slice(20, 30)),
    ],
    dim=["x", "y"],  # FIXME: support dim="x"?
)

In [ ]:
newds.xindexes["x"]._wrapped_indexes

In [ ]:
newds = with_geoindex(ds)
newds

In [ ]:
newds.isel(x=slice(None, None, 2), y=slice(None, None, 2))

In [ ]:
newds.isel(x=slice(10, 20))

In [ ]:
newds.sel(x=[120, 121], y=[-88.5, -88], options=dict(crs="EPSG:4326"))

In [ ]:
# Before I added isel
newds.sel(xc=[120, 121], yc=[-88.5, -88], options=dict(crs="EPSG:4326"))